In [1]:
from n2v.models import N2V
import numpy as np
from csbdeep.io import save_tiff_imagej_compatible
from n2v.internals.N2V_DataGenerator import N2V_DataGenerator

2024-06-05 10:30:56.445453: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-05 10:30:56.698189: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-05 10:30:56.699797: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 10:30:57.624315: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:

dome_path = './../../../Desktop/dome32_tp3/' # Fill with the path of the dome
name_of_dome = 't3-top-1.tif' # Fill with the name of the dome
save_dome_path = './../../../Desktop/dome32_tp3/' # Fill with the path to save the dome
name_save_dome = name_of_dome.split('.')[0]+'_denoised.tif'

dims = 'ZYX'

In [4]:
datagen = N2V_DataGenerator()
img = datagen.load_imgs_from_directory(directory=dome_path, filter=name_of_dome, dims=dims)

print(img[0].shape)

(1, 1086, 1304, 1304, 1)


In [5]:
model_name = '2d_images_from_stack'
basedir = './n2v_2d_black_line/models_test/'
model = N2V(config=None, name=model_name, basedir=basedir)

2024-06-05 10:32:31.430654: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-05 10:32:31.431365: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Loading network weights from 'weights_best.h5'.


In [6]:
def denoise2dSlice(img):
    image = img[:, :, 0]
    # print(image.shape)

    non_zero_cols = np.any(image != 0, axis = 0)
    first_non_zero_col = np.argmax(non_zero_cols)
    last_non_zero_col = len(non_zero_cols) - np.argmax(non_zero_cols[::-1]) - 1
    clean_image = image[:, first_non_zero_col:last_non_zero_col + 1]

    pred = model.predict(clean_image, axes='YX')
    full_pred = np.zeros_like(image)
    full_pred[:, first_non_zero_col:last_non_zero_col + 1] = pred

    return full_pred

def denoise3d(img):
    stack = []
    for i in img:
        stack.append(denoise2dSlice(i))
    stack = np.array(stack)
    return stack

In [8]:
img = img[0]
stack = []

if len(dims) == 2:
    stack = denoise2dSlice(img=img)
if len(dims) == 3:
    stack = denoise3d(img=img)
if len(dims) == 4:
    for i in img:
        stack.append(denoise3d(img=i))
stack = np.array(stack)

2024-06-05 10:33:08.280673: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 435306496 exceeds 10% of free system memory.
2024-06-05 10:33:08.336507: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 435306496 exceeds 10% of free system memory.
2024-06-05 10:33:08.427106: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 435306496 exceeds 10% of free system memory.
2024-06-05 10:33:08.607233: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 435306496 exceeds 10% of free system memory.
2024-06-05 10:33:09.753489: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 435306496 exceeds 10% of free system memory.


1/1 [==============================] - 3s 3s/step


In [9]:
save_tiff_imagej_compatible(str(save_dome_path)+str(name_save_dome), stack, axes=dims)

/home/ppujol@ibec.local/python_codes/light_sheet_domes/venv_tf/lib/python3.10/site-packages/tifffile/tifffile.py:3776: UserWarning: <tifffile.TiffWriter 't3-top-1_denoised.tif'> truncating ImageJ file
  warnings.warn(
